In [1]:
import tensorflow as tf
from utils.image import load_image
from utils.image import affine_transform
from utils.image import lens_distortion
import matplotlib.pyplot as plt
import numpy as np

tf.enable_eager_execution()

In [19]:
image = load_image('./data/montgomery/images/MCUCXR_0001_0.png')
left_mask = load_image('./data/montgomery/masks/left/MCUCXR_0001_0.png')
right_mask = load_image('./data/montgomery/masks/right/MCUCXR_0001_0.png')
mask = tf.logical_or(tf.greater(left_mask,0.5), tf.greater(right_mask,0.5))
mask = tf.cast(mask, tf.float32)

# image = lens_distortion(image, k=0)
image = affine_transform(image, rot=np.pi/12)
mask = affine_transform(mask, rot=np.pi/12)

image = tf.tile(image, [1, 1, 3])
tf.assign_add(image[:,:,2],mask)

print(image.shape)
plt.imshow(tf.cast(tf.squeeze(image)*255, tf.int32))
plt.show()

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign_add'

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import random

tf.enable_eager_execution()

In [ ]:
def load_image(path):
    # Read raw image (string of pixel values)
    image = tf.read_file(path)
    
    # Decode the raw image into pixel array
#     image = tf.image.decode_image(image)    # This is supposed to do the same thing as below, but there seems to be a bug...
    image = tf.cond(tf.image.is_jpeg(image),
      lambda: tf.image.decode_jpeg(image),
      lambda: tf.image.decode_png(image))
    
#     image = tf.image.decode_image(image)   # Decode the raw image into pixel array
    image = tf.cast(image, tf.float32)     # Make sure the pixel values are in floating point numbers, NOT INTEGERS
    image = tf.image.resize(image, [384, 384])
    image /= 255.0
    
    return image

def linear_transform(n=1, scale=(1,1), angle=0, shear=(0,0), translate=(0,0)):
    M = np.zeros((3,3,1))
    M[:,0,:] = translate[:,0]
    M = [[0,0,translate(0)], [0,0,translate(1)], [0,0,1]]
    M *= [[cos(angle), -sin(angle), 0],[sin(angle), cos(angle), 0],[0,0,1]]
    M *= [[scale[0], 0, 0],[0, scale[0], 0],[0,0,1]]
    M *= [[1, shear[0], 0],[shear[1], 1, 0],[0,0,1]]
    return M

# tf.contrib.image.transform(image)
    

In [ ]:
n=10
translate = np.array([np.arange(n), np.arange(n,2*n)])
T = np.zeros((n,3,3))
T[:,0,0] = 1
T[:,1,1] = 1
T[:,2,2] = 1
T[:,0,2] = translate[0,:]
T[:,1,2] = translate[1,:]

angle = np.linspace(-np.pi/6, np.pi/6, num=n)
R = np.zeros((n,3,3))
R[:,2,2] = 1;
R[:,0,0] = np.cos(angle)
R[:,1,1] = np.cos(angle)
R[:,0,1] = -np.sin(angle)
R[:,1,0] = np.sin(angle)

M = np.matmul(T, R)

In [ ]:
R[1,:,:]

In [ ]:
image = load_image('./data/montgomery/images/MCUCXR_0001_0.png')
print(image.shape)
plt.imshow(np.squeeze(image), cmap='gray')

In [ ]:
import random
montgomery_root = pathlib.Path('./data/montgomery')

# read image paths using glob and convert them into string format
montgomery_image_paths = [str(path) for path in list(montgomery_root.glob('images/*.png'))]
montgomery_mask_left_paths = [str(path) for path in list(montgomery_root.glob('masks/left/*.png'))]
montgomery_mask_right_paths = [str(path) for path in list(montgomery_root.glob('masks/right/*.png'))]

# shuffle them in a random order
temp = list(zip(montgomery_image_paths, montgomery_mask_left_paths, montgomery_mask_right_paths)) # zip the three paths to make sure they are shuffled together
random.shuffle(temp) # shuffle
montgomery_image_paths, montgomery_mask_left_paths, montgomery_mask_right_paths = zip(*temp) # unzip them
montgomery_image_paths = list(montgomery_image_paths)           # unzipping converts the lists to tuples for some reason...
montgomery_mask_left_paths = list(montgomery_mask_left_paths)   # we are explicitly converting them back to lists
montgomery_mask_right_paths = list(montgomery_mask_right_paths) #

In [ ]:
montgomery_image_path_ds = tf.data.Dataset.from_tensor_slices(montgomery_image_paths)
montgomery_mask_left_path_ds = tf.data.Dataset.from_tensor_slices(montgomery_mask_left_paths)
montgomery_mask_right_path_ds = tf.data.Dataset.from_tensor_slices(montgomery_mask_right_paths)

print('shape: ', repr(montgomery_image_path_ds.output_shapes))
print('type: ', montgomery_image_path_ds.output_types)
print()
print(montgomery_image_path_ds)
print()

montgomery_image_ds = montgomery_image_path_ds.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
import IPython.display as display
for n in range(3):
  image_path = random.choice(montgomery_image_paths)
  image = load_image(image_path)
  display.display(display.Image(image))

In [ ]:
for path in montgomery_image_path_ds.take(15):
    print(path)

In [ ]:
import pathlib
data_root_orig = tf.keras.utils.get_file('flower_photos',
                                         'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

In [ ]:
for item in data_root.iterdir():
  print(item)

In [ ]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

In [ ]:
all_image_paths[:10]

In [ ]:
import os
attributions = (data_root/"LICENSE.txt").open(encoding='utf-8').readlines()[4:]
attributions = [line.split(' CC-BY') for line in attributions]
attributions = dict(attributions)

In [ ]:
import IPython.display as display

def caption_image(image_path):
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    return "Image (CC BY 2.0) " + ' - '.join(attributions[str(image_rel)].split(' - ')[:-1])

In [ ]:
for n in range(3):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(caption_image(image_path))
  print()

In [ ]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

In [ ]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

In [ ]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

In [ ]:
img_path = all_image_paths[0]
img_path

In [ ]:
img_raw = tf.read_file(img_path)
print(repr(img_raw)[:100]+"...")

In [ ]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range

  return image

In [ ]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

In [ ]:
import matplotlib.pyplot as plt

img_path = all_image_paths[0]
label = all_image_labels[0]

plt.imshow(load_and_preprocess_image(img_path))
plt.grid(False)
plt.xlabel(caption_image(img_path).encode('utf-8'))
plt.title(label_names[label].title())
print()

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
print('shape: ', repr(path_ds.output_shapes))
print('type: ', path_ds.output_types)
print()
print(path_ds)

In [ ]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
for n,image in enumerate(image_ds.take(4)):
  plt.subplot(2,2,n+1)
  plt.imshow(image)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.show()

In [ ]:
import pathlib
data_root = pathlib.Path('./data')
print(data_root)

In [ ]:
for item in data_root.iterdir():
    print(item)

In [ ]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

In [ ]:
data_root.glob('*/*')

In [ ]:
path = './data/montgomery/images/MCUCXR_0001_0.png'
image = tf.io.read_file(path)


In [ ]:
img_tensor = tf.image.decode_image(image)
print(img_tensor.shape)

In [ ]:
np.max(img_tensor)